# 3. OpenSearch Hands-On (Part2)

## 3.1. OpenSearch との接続

In [1]:
import pprint
from opensearchpy import OpenSearch

client = OpenSearch(
    hosts=[{"host": "localhost", "port": 9200}],
    http_compress=True,  # 圧縮有効化
)

# クラスタ情報の確認
pprint.pprint(client.info())

{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'u_s80VD9RSCBTiz2OAThVg',
 'name': '9e342b1d841b',
 'tagline': 'The OpenSearch Project: https://opensearch.org/',
 'version': {'build_date': '2024-10-31T19:08:04.231254959Z',
             'build_hash': '99a9a81da366173b0c2b963b26ea92e15ef34547',
             'build_snapshot': False,
             'build_type': 'tar',
             'distribution': 'opensearch',
             'lucene_version': '9.12.0',
             'minimum_index_compatibility_version': '7.0.0',
             'minimum_wire_compatibility_version': '7.10.0',
             'number': '2.18.0'}}


## 3.2. インデックスの作成

### 3.2.1 インデックスを明示的に作成する理由

- 適切な **インデックスマッピング（データ型設定）** を行うためには、**事前にインデックスを作成** するのがベストです。  
- 以下のように、スクリプトの最初で `client.indices.create()` を使用して **インデックスを明示的に作成** しましょう。





#### なぜインデックスを明示的に作成すべきか？

1. データ型を正しく指定できる
	- デフォルトではすべてのフィールドが `text` 型になる
		- `keyword` 型として扱うべき `file_name` や `sheet_name` が正しく検索できなくなる。
	- `row` と `column` も **整数型（integer）** にしておくと、数値での検索やソートが容易になる。
2. パフォーマンス向上
   - `keyword` 型を使うことで、ファイル名・シート名の **フィルタリング（exact match）** が高速になる。
3. フィールドの意図を明確にできる
   - どのフィールドを全文検索に使うのか（`text`）または、
   - どのフィールドを識別情報に使うのか（`keyword`）を明確にできる。

#### 最終的なコードの流れ

1. `create_index()` を実行し、OpenSearch に `excel_data` インデックスを作成
2. `process_unstructured_excel(directory_path)` を実行し、Excelの非構造データをインデックス化
3. `search_data()` などを用いて、検索＆表示

#### 補足

- OpenSearch にデータを送信すると、**インデックスが存在しない場合は自動作成** されます。
- **ただし、自動作成されたインデックスのデフォルトマッピングでは、全てのフィールドが `text` になる可能性があるため、適切な設定を行うべき** です。

In [2]:
# OpenSearchの設定
OPENSEARCH_HOST = "http://localhost:9200"
INDEX_NAME = "excel_data"

In [3]:
# OpenSearchにインデックスを作成する関数
def create_index():
    """OpenSearchのインデックスを作成（存在しない場合のみ）"""
    index_body = {
        "settings": {
            "index": {
                "number_of_shards": 1,
                "number_of_replicas": 0
            }
        },
        "mappings": {
            "properties": {
                "file_name": {"type": "keyword"},  	# Excelファイル名
                "sheet_name": {"type": "keyword"},  # シート名
                "row": {"type": "integer"},  		# Excelの行番号
                "column": {"type": "integer"},  	# Excelの列番号
                "text": {"type": "text"}  			# セルの内容（全文検索可能）
            }
        }
    }

    # インデックスが既に存在するかチェック
    if not client.indices.exists(INDEX_NAME):
        client.indices.create(index=INDEX_NAME, body=index_body)
        print(f"インデックス '{INDEX_NAME}' を作成しました。")
    else:
        print(f"インデックス '{INDEX_NAME}' は既に存在します。")

# インデックスを作成（最初に実行）
create_index()

インデックス 'excel_data' を作成しました。


## 3.3. エクセルデータの取り込み

In [4]:
import os
import openpyxl
from opensearchpy import OpenSearch, helpers


client = OpenSearch(
    hosts=[OPENSEARCH_HOST],
    http_auth=('admin', 'myP@ssw0rdForOpensearch'),  # 認証情報（環境に応じて変更）
)

# Excelファイルの読み込み（構造化されていないデータ対応）
def read_unstructured_excel(file_path):
    """Excelファイルをセル単位で読み込み、JSON化"""
    wb = openpyxl.load_workbook(file_path)
    data = []
    
    for sheet in wb.sheetnames:
        ws = wb[sheet]
        
        for row in ws.iter_rows():
            for cell in row:
                if cell.value:  # 空白セルを除外
                    data.append({
                        "file_name": os.path.basename(file_path),
                        "sheet_name": sheet,
                        "row": cell.row,
                        "column": cell.column,
                        "text": str(cell.value)
                    })
    
    return data

# OpenSearchにデータを一括登録
def bulk_index_data(data):
    """OpenSearchにデータを登録"""
    actions = [
        {
            "_index": INDEX_NAME,
            "_source": record
        }
        for record in data
    ]
    
    helpers.bulk(client, actions)

# 指定されたディレクトリ内のすべてのExcelファイルを処理し、OpenSearchに登録する
def process_unstructured_excel(directory_path):
    all_exel_files = []  # 全てのExcelファイルのデータを格納するリスト

    # 指定されたディレクトリ内のファイルを一覧取得
    for file_name in os.listdir(directory_path):  
        # ファイルがExcel形式（.xlsx または .xls）かをチェック
        if file_name.endswith(".xlsx") or file_name.endswith(".xls"):
            file_path = os.path.join(directory_path, file_name)  # フルパスを作成
            print(f"Processing: {file_name}")  # 現在処理中のファイルを出力
            
            # Excelファイルのデータを読み込み、リストに追加
            all_exel_files.extend(read_unstructured_excel(file_path))  

    # 収集したデータが1つ以上ある場合、OpenSearchに登録
    if all_exel_files:
        bulk_index_data(all_exel_files)  # データを一括登録
        print("データの登録が完了しました。")  # 登録完了のメッセージを出力


In [5]:
# 実行
directory_path = "datasets"  # Excelファイルがあるフォルダ
process_unstructured_excel(directory_path)

Processing: 家計簿2.xlsx
Processing: 家計簿1.xlsx
データの登録が完了しました。


/Users/daisuke/03_a-labs/venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


## 3.4. エクセルデータの検索

In [6]:
# 検索関数（全文検索対応）
def search_data(query_text):
    """OpenSearchの全文検索"""
    query = {
        "query": {
            "match": {
                "text": query_text
            }
        }
    }
    
    response = client.search(index=INDEX_NAME, body=query)
    return response["hits"]["hits"]


In [7]:
# 検索例
keyword = "インターネット"
results = search_data(keyword)
for res in results:
    pprint.pprint(res["_source"])

{'column': 2,
 'file_name': '家計簿2.xlsx',
 'row': 33,
 'sheet_name': 'Jan',
 'text': 'インターネット'}
{'column': 2,
 'file_name': '家計簿2.xlsx',
 'row': 33,
 'sheet_name': 'Feb',
 'text': 'インターネット'}
{'column': 2,
 'file_name': '家計簿1.xlsx',
 'row': 33,
 'sheet_name': 'Jan',
 'text': 'インターネット'}
{'column': 2,
 'file_name': '家計簿1.xlsx',
 'row': 33,
 'sheet_name': 'Feb',
 'text': 'インターネット'}


In [8]:
# 最初の一つ目だけ、全データを表示してみる
results[0]

{'_index': 'excel_data',
 '_id': 'bUuD9JQBK1tsZIgq57G_',
 '_score': 6.91768,
 '_source': {'file_name': '家計簿2.xlsx',
  'sheet_name': 'Jan',
  'row': 33,
  'column': 2,
  'text': 'インターネット'}}

# Dashboardの使い方
**OpenSearch Dashboards から Python で作成したインデックスを確認する方法**を説明します。

---

## **1. Dashboards にアクセス**
ブラウザで **`http://localhost:5601`** にアクセスし、OpenSearch Dashboards を開きます。

---

## **2. インデックスが登録されているか確認**
1. **[Dev Tools] を開く**
   - 左側のメニューから `Dev Tools` を選択。
   - または、URL に直接 `http://localhost:5601/app/dev_tools` を入力。

2. **インデックス一覧を確認**
   - `Console` に以下のクエリを入力し、**[▶︎実行] ボタンをクリック**。
   ```json
   GET _cat/indices?v
   ```
   - Python で作成したインデックス名がリストに表示されているか確認。

---

## **3. インデックスのデータを確認**
1. **登録データのプレビュー**
   - `Console` に以下を入力し、**[▶︎実行]**。
   ```json
   GET your_index_name/_search?pretty
   ```
   - `your_index_name` は Python で作成したインデックス名に置き換えてください。

   - 例：
     ```json
     GET test-index/_search?pretty
     ```

   - 登録されたデータが `hits.hits` 配列内に表示される。

---

## **4. インデックスパターンを作成**
データを可視化するには、インデックスパターンを作成する必要があります。

### **4.1 [管理] 画面でインデックスパターンを作成**
1. **[Stack Management] に移動**
   - 左側のメニューから `Stack Management` をクリック。
   - または `http://localhost:5601/app/management` にアクセス。

2. **[Index Patterns] を選択**
   - `Index Patterns` メニューを開く。

3. **[Create index pattern] をクリック**
   - インデックス名（例: `test-index*`）を入力し、**Next step** をクリック。
   - `Timestamp field` を選択（時系列データの場合）。
   - **[Create index pattern] をクリック**。

---

## **5. 可視化 (Visualization)**
1. **[Discover] に移動**
   - 左側のメニューから `Discover` を選択。
   - 作成したインデックスパターンを選択。
   - 登録データが表示されるか確認。

2. **[Visualize Library] でグラフを作成**
   - `Visualize Library` を選択。
   - `Create visualization` → `Bar chart`, `Line chart` などを選択。
   - `Index pattern` に作成したパターンを選択し、グラフを作成。

---

## **Dashboardの使い方まとめ**
| 手順 | 操作 | URL |
|------|------|------|
| 1. Dashboards にアクセス | `http://localhost:5601` | OpenSearch Dashboards を開く |
| 2. インデックスを確認 | `GET _cat/indices?v` | `Dev Tools` で一覧表示 |
| 3. データを確認 | `GET your_index_name/_search?pretty` | `Dev Tools` で検索 |
| 4. インデックスパターンを作成 | `Stack Management > Index Patterns` | `http://localhost:5601/app/management` |
| 5. データを可視化 | `Discover` または `Visualize Library` | `http://localhost:5601/app/discover` |

---

これで、Python で登録したデータを OpenSearch Dashboards から可視化できます！🚀

## [補足] Dashboardにおける検索結果の表示件数を制御する方法

`GET your_index_name/_search?pretty` を実行した際に、すべてのデータが表示されない場合、OpenSearch のデフォルト設定では **最大 10 件** しか表示されません。

| 方法 | クエリ | 備考 |
|------|------|------|
| **全件取得** | `GET your_index_name/_search?size=10000` | 最大 10,000 件まで |
| **総件数を確認** | `GET your_index_name/_count` | データが何件あるか確認 |
| **ページネーション** | `GET your_index_name/_search?size=100&from=100` | 100 件ずつ取得 |
| **大量データ取得** | `POST your_index_name/_search?scroll=1m` | Scroll API を利用 |

この方法で、Python から登録したすべてのデータを取得できます！🚀

In [9]:
# 作成したインデックスを削除
response = client.indices.delete(
    index=INDEX_NAME
)

# インデックス削除の結果を出力
pprint.pprint(response)

{'acknowledged': True}
